In [ ]:
#리서치 보고서
import requests
from bs4 import BeautifulSoup as bs

def get_detail_div(base_url, nm):
    response = requests.get( base_url, headers={"User-agent": "Mozilla/5.0"} )
    soup = bs(response.text, 'html.parser')
    desc = ""

    print(nm)
    for row in soup.find("table",{"summary":"종목분석 리포트 본문내용"}).find_all("div"):
        line = str(row)
        if 'div style' not in line: continue
        try:
            desc += line.split("<b>")[1]
        except:
            print(line)
    print(desc)

    return desc

def get_detail(base_url, nm):
    response = requests.get( base_url, headers={"User-agent": "Mozilla/5.0"} )
    soup = bs(response.text, 'html.parser')
    desc = ""
    for row in soup.find("table",{"summary":"종목분석 리포트 본문내용"}).find_all("p"):
        line = str(row)
        if '<p class="source">' in line:
            continue
        desc += line.replace("<p>","").replace("</p>","").replace("<strong>","").replace("</strong>","").replace("<br/>","") + "\n"
    return desc

base_url = f"https://finance.naver.com/research/company_list.nhn"
response = requests.get( base_url, headers={"User-agent": "Mozilla/5.0"} )
soup = bs(response.text, 'html.parser')
idx = 0

list_cols = ["JONGMOK_CD", "JONGMOK_NM", "HREF", "TITLE"]
list_headline = []
list_line = []
for row in soup.find("table",{"summary":"종목분석 리포트 게시판 글목록"}).find_all("td"):
    line = str(row)
    if 'class="file"' in line: continue
    if "stock_item" in line:
        cd = "A" + line.split("code=")[1].split(" ")[0].replace('"',"")
        nm = line.split("title=")[1].split(">")[0].replace("</a>","").replace('"',"")
        list_line.append(cd)
        list_line.append(nm)
    elif "href" in line:
        href = 'https://finance.naver.com/research/' + line.split("href=")[1].split(">")[0].replace("amp;","").replace('"',"")
        title = line.split("href=")[1].split(">")[1].replace("</a></td>","").replace("</a","")
        list_line.append(href)
        list_line.append(title)
        list_headline.append(list_line)
        list_line = []
    else:
        continue

list_result = []
for list_detail in list_headline:
    desc = get_detail(list_detail[2], list_detail[1])
    list_result.append(list_detail[0])
    list_result.append(list_detail[1])
    desc = list_detail[3] + " " + desc.replace("\xa0","").replace("\n"," ")
    list_result.append(desc.replace('amp;','').replace("..",""))
    print(list_result)
    list_result = []